In [ ]:
import kineticstoolkit.lab as ktk
import pandas as pd
# Set an interactive backend, not required if already enabled in Spyder
%matplotlib qt5
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import roc_curve, roc_auc_score
from pyopls import OPLS
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict, LeaveOneOut
from sklearn.metrics import r2_score, accuracy_score

In [ ]:
# Define the base path
base_path = 'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data'

# Specify the list of subjects you want to load
subjects = ["Subject01", "Subject02", "Subject03"]

# Initialize an empty list to hold each subject's data
data_frames = []

# Loop through each subject, load the CSV file, and append it to the list
for subject in subjects:
    file_path = os.path.join(base_path, f"{subject}_rotation_analysis.csv")
    df = pd.read_csv(file_path)
    df['Subject'] = subject  # Add a column for the subject
    data_frames.append(df)

# Concatenate all the data into a single DataFrame
combined_data = pd.concat(data_frames, ignore_index=True)

# Save the combined data to a new CSV file
combined_data_path = os.path.join(base_path, "all_subjects_rotation_analysis.csv")
combined_data.to_csv(combined_data_path, index=False)


Combined data saved to C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data\all_subjects_rotation_analysis.csv


In [179]:
combined_data

,subject_id,early_det_value,late_det_value,pseudo_efficiency,pseudo_efficiency_late,early_num_pcs_90,early_num_pcs_above_1_percent,late_num_pcs_90,late_num_pcs_above_1_percent,Subject
0,Subject01,9.312549e+44,2.951651e+42,4.227450e+08,7.203102e+09,4,7,3,7,Subject01
1,Subject02,4.261045e+39,8.979162e+51,9.952271e+09,5.234196e+11,5,8,6,9,Subject02
2,Subject03,1.545710e+41,2.732627e+55,3.706173e+08,4.425569e+09,7,11,9,13,Subject03


# Measure across subjects

In [193]:
num_subjects = 3
conditions = ["Early", "Late"]

# Apply log transformation to the data for convenience in display
subject_data = {
    "Efficiency": [
        np.log10(combined_data['pseudo_efficiency']),
        np.log10(combined_data['pseudo_efficiency_late'])
    ],
    "Complexity": [
        combined_data['early_num_pcs_above_1_percent'],
        combined_data['late_num_pcs_above_1_percent']
        # combined_data['early_num_pcs_90'],
        # combined_data['late_num_pcs_90']
    ]
}

# Calculate means and standard deviations
means = {key: [np.mean(subject_data[key][0]), np.mean(subject_data[key][1])] for key in subject_data}
std_devs = {key: [np.std(subject_data[key][0]), np.std(subject_data[key][1])] for key in subject_data}


In [204]:
# Plot setup
fig, axs = plt.subplots(1, 2, figsize=(6, 3))  # Smaller, narrower figure size
plt.tight_layout(pad=2)  # Add a bit of padding for clarity

# fig.suptitle("Simulated Data for Movement Efficiency and Complexity")

# Colors
early_color = 'blue'
late_color = 'red'
line_color = (0.8, 0.6, 0.8)

# Plot for each panel
for idx, (key, ax) in enumerate(zip(subject_data.keys(), axs)):
    # Get data for each condition
    early_means = means[key][0]
    late_means = means[key][1]
    early_sem = std_devs[key][0] / np.sqrt(num_subjects)
    late_sem = std_devs[key][1] / np.sqrt(num_subjects)

    # Replace bar plot with mean points and error bars
    ax.errorbar(conditions[0], early_means, yerr=early_sem, fmt='o', color=early_color, capsize=5)
    ax.errorbar(conditions[1], late_means, yerr=late_sem, fmt='o', color=late_color, capsize=5)

    # Plot subject lines/scatter
    for i in range(num_subjects):
        ax.plot(conditions, [subject_data[key][0][i], subject_data[key][1][i]],
                color=line_color, linestyle='--', marker='.', markersize=10)
        # Create scatter plot for each subject in early and late conditions
        # ax.scatter(conditions, [subject_data[key][0][i], subject_data[key][1][i]],
        #        color=line_color, marker='.', s=15)  # s is used for marker size


    # Title and labels
    ax.set_title(f'Movement {key}')
    ax.set_xlabel('Adaptation Phase')
    # ax.set_ylabel(f'{key}')
    ax.set_ylim(0, max(late_means + late_sem, early_means + early_sem) + 5)
    ax.set_xlim(-0.2, 1.2)
    ax.text(-0.2, 1.1, chr(65 + idx), transform=ax.transAxes, fontsize=12, fontweight='bold', ha='center', va='center')

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.3)

# Show the plot
plt.tight_layout()

plt.savefig('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/plot/3subjects.png', bbox_inches='tight', dpi=600)


# Simple angle plot

In [ ]:
# Define the path to the CSV file to store the early data
Subejct01_alljoints_path = os.path.join('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/', "Subejct01_all_joints_df.csv")
# Save the all_fish_df to the CSV file
Subject01_alljoints_data = pd.read_csv(Subejct01_alljoints_path)
Subejct01_rotation_path = os.path.join('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/', "Subejct01_angerror_df.csv")
Subject01_rotation_angles= pd.read_csv(Subejct01_rotation_path)

# Define the path to the CSV file to store the early data
Subejct02_alljoints_path = os.path.join('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/', "Subejct02_all_joints_df.csv")
# Save the all_fish_df to the CSV file
Subject02_alljoints_data = pd.read_csv(Subejct02_alljoints_path)
Subejct02_rotation_path = os.path.join('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/', "Subejct02_angerror_df.csv")
Subject02_rotation_angles= pd.read_csv(Subejct02_rotation_path)


# Define the path to the CSV file to store the early data
Subejct03_alljoints_path = os.path.join('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/', "Subejct03_all_joints_df.csv")
# Save the all_fish_df to the CSV file
Subject03_alljoints_data = pd.read_csv(Subejct03_alljoints_path)
Subejct03_rotation_path = os.path.join('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/', "Subejct03_angerror_df.csv")
Subject03_rotation_angles= pd.read_csv(Subejct03_rotation_path)

In [405]:
# Specify the subject ID dynamically
subject_id = "Subject03"

# Load data dynamically for the specified subject
alljoints_data = globals()[f"{subject_id}_alljoints_data"]
rotation_angles = globals()[f"{subject_id}_rotation_angles"]

# Extract early and late rotation data
Rotation_data_early = alljoints_data[(alljoints_data['Trial ID'] >= 11) & (alljoints_data['Trial ID'] <= 310)]
# Rotation_data_late = alljoints_data[(alljoints_data['Trial ID'] >= 211) & (alljoints_data['Trial ID'] <= 310)]

# Select elements for early and late angle errors
early_angerror = rotation_angles[10:310]
# late_angerror = rotation_angles[210:310]


OPLS_NoP = 25

In [406]:
early_angerror = np.array(early_angerror)
# late_angerror = np.array(late_angerror)

In [407]:
# # Assuming early_angerror and late_angerror are arrays or lists with 100 elements each.
# # Replace `x_values` with your actual x-coordinates if they are defined. If you just want indices as x-values:
# x_values = range(300)
# plt.figure(figsize=(10, 6))
# # Scatter plot for early_angerror
# plt.scatter(x_values, early_angerror, label="Early Angle Error", color='blue')

# # # Scatter plot for late_angerror
# # plt.scatter(x_values, late_angerror, label="Late Angle Error", color='red')

# # Adding labels and title
# plt.xlabel("Index")
# plt.ylabel("Angle Error")
# plt.title("Scatter Plot of Early and Late Angle Errors")

# # Show legend
# plt.legend()

# # Display the plot
# plt.show()

In [408]:
# First, ensure the 'Angle Name' and 'Trial ID' are treated as categorical values
Rotation_data_early['Angle Name'] = Rotation_data_early['Angle Name'].astype('category')
# Rotation_data_late ['Angle Name'] = Rotation_data_late ['Angle Name'].astype('category')

# Loop through the unique 'Angle Name' values and create a new CSV for each angle
for angle_name in Rotation_data_early['Angle Name'].unique():
    # Create a subset for the current 'Angle Name'
    subset = Rotation_data_early[Rotation_data_early['Angle Name'] == angle_name].copy()  # Use .copy() to avoid the warning
    
    # Ensure early_angerror is the same length as the subset, or repeat/adjust it accordingly
    if len(early_angerror) == len(subset):
        subset.loc[:, 'angerror'] = early_angerror  # Use .loc to avoid setting on a slice
    else:
        # If lengths don't match, repeat early_angerror to match the number of rows in the subset
        print(f"Warning: Length mismatch for angle {angle_name}. Adjusting the 'angerror' length.")
        repeated_angerror = np.tile(early_angerror, len(subset) // len(early_angerror) + 1)[:len(subset)]
        subset.loc[:, 'angerror'] = repeated_angerror

    # Save the subset to a CSV file, using the angle name in the file name
    file_name = f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_early_{angle_name}.csv'
    subset.to_csv(file_name, index=False)


# # Loop through the unique 'Angle Name' values and create a new CSV for each angle
# for angle_name in Rotation_data_late['Angle Name'].unique():
#     subset = Rotation_data_late[Rotation_data_late['Angle Name'] == angle_name].copy() 

#     # Ensure late_angerror is the same length as the subset, or repeat/adjust it accordingly
#     if len(late_angerror) == len(subset):
#         subset.loc[:, 'angerror'] = late_angerror  # Use .loc to avoid setting on a slice
#     else:
#         # If lengths don't match, repeat early_angerror to match the number of rows in the subset
#         print(f"Warning: Length mismatch for angle {angle_name}. Adjusting the 'angerror' length.")
#         repeated_angerror = np.tile(late_angerror, len(subset) // len(late_angerror) + 1)[:len(subset)]
#         subset.loc[:, 'angerror'] = repeated_angerror

    
    # Save the subset to a CSV file, using the angle name in the file name
    file_name = f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_late_{angle_name}.csv'
    subset.to_csv(file_name, index=False)

# for angle_name in Rotation_data_late['Angle Name'].unique():
#     subset = Rotation_data_late [Rotation_data_late ['Angle Name'] == angle_name]
    

SettingWithCopyWarning [C:\Users\86153\AppData\Local\Temp\ipykernel_31184\2118411671.py:2] 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [409]:
foearm_early_path =f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_early_R_Forearm[0] max vel.csv'
elbow_early_path = f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_early_R_Forearm[2] max vel.csv'
hand_early_path = f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_early_R_Hand[1] max vel.csv'

arm1_early_path = f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_early_R_Arm[1] max vel.csv'
arm2_early_path = f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_early_R_Arm[2] max vel.csv'


# foearm_late_path =f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_late_R_Forearm[0] max vel.csv'
# elbow_late_path = f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_late_R_Forearm[2] max vel.csv'
# hand_late_path = f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/Rotation_data_late_R_Hand[1] max vel.csv'

foearm_early = pd.read_csv(foearm_early_path)
elbow_early = pd.read_csv(elbow_early_path)
hand_early = pd.read_csv(hand_early_path)
arm1_early = pd.read_csv(arm1_early_path)
arm2_early = pd.read_csv(arm2_early_path)

# foearm_late = pd.read_csv(foearm_late_path)
# elbow_late = pd.read_csv(elbow_late_path)
# hand_late = pd.read_csv(hand_late_path)

In [410]:
# List of DataFrames and names for reference
dataframes = {
    "foearm_early": foearm_early,
    "elbow_early": elbow_early,
    "hand_early": hand_early,
    "arm1_early": arm1_early,
    "arm2_early": arm2_early
    # "foearm_late": foearm_late,
    # "elbow_late": elbow_late,
    # "hand_late": hand_late
}
# Initialize a list to store results
results = []

# Iterate through each DataFrame and calculate the mean SUC angle
for name, df in dataframes.items():
    Sucangle = 0
    i = 0
    
    # Loop through each row of the current DataFrame
    for index, row in df.iterrows():
        # Check if 'angerror' is between -1 and 1
        if -1 <= row['angerror'] <= 1:
            # Add the 'Euler' value to the running total
            Sucangle += row['Euler']
            # Increment the counter
            i += 1
    
    # Calculate the mean Euler angle for valid rows
    if i > 0:
        meanSUCangle = Sucangle / i
    else:
        meanSUCangle = None  # If no valid rows, set mean to None

    # Append the result to the list with the DataFrame name and meanSUCangle
    results.append({"DataFrame": name, "Mean_SUC_Angle": meanSUCangle})

# Convert the results to a DataFrame
mean_suc_df = pd.DataFrame(results)

# Display or save the DataFrame for later analysis
print(mean_suc_df)

# Optionally, save to a CSV file
mean_suc_df.to_csv(f'C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/data/{subject_id}/mean_suc_angles.csv', index=False)

      DataFrame  Mean_SUC_Angle
0  foearm_early       66.707986
1   elbow_early      -74.643491
2    hand_early      -26.851469
3    arm1_early       -2.492924
4    arm2_early      -41.202304


In [411]:
# # Plot 'Euler' values for 'foearm_early' and 'foearm_late', adjusted by the mean SUC angle for 'foearm'
# plt.figure(figsize=(10, 6))

# # Scatter plot for foearm_early
# plt.scatter(foearm_early['Trial ID'], foearm_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][0], label='Forearm Early', color='blue', alpha=0.7)

# # # Scatter plot for foearm_late
# # plt.scatter(foearm_late['Trial ID'], foearm_late['Euler'] - mean_suc_df['Mean_SUC_Angle'][0], label='Forearm Late', color='green', alpha=0.7)

# # Add a purple horizontal line at y = 0
# plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# # Add title, labels, and legend
# plt.title("Forearm: Euler Angles Adjusted by Mean SUC Angle")
# plt.xlabel("Trial ID")
# plt.ylabel("Adjusted Euler Angle")
# plt.legend()

# # Display the plot
# plt.show()

In [412]:
# # Plot 'Euler' values for 'elbow_early' and 'elbow_late', adjusted by the mean SUC angle for 'elbow'
# plt.figure(figsize=(10, 6))

# # Scatter plot for elbow_early
# plt.scatter(elbow_early['Trial ID'], elbow_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][1], label='Elbow Early', color='blue', alpha=0.7)

# # # Scatter plot for elbow_late
# # plt.scatter(elbow_late['Trial ID'], elbow_late['Euler'] - mean_suc_df['Mean_SUC_Angle'][1], label='Elbow Late', color='green', alpha=0.7)

# # Add a purple horizontal line at y = 0
# plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# # Add title, labels, and legend
# plt.title("Elbow: Euler Angles Adjusted by Mean SUC Angle")
# plt.xlabel("Trial ID")
# plt.ylabel("Adjusted Euler Angle")
# plt.legend()

# # Display the plot
# plt.show()


In [413]:
# # Plot 'Euler' values for 'hand_early' and 'hand_late', adjusted by the mean SUC angle for 'hand'
# plt.figure(figsize=(10, 6))

# # Scatter plot for hand_early
# plt.scatter(hand_early['Trial ID'], hand_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][2], label='Hand Early', color='blue', alpha=0.7)

# # # Scatter plot for hand_late
# # plt.scatter(hand_late['Trial ID'], hand_late['Euler'] - mean_suc_df['Mean_SUC_Angle'][2], label='Hand Late', color='green', alpha=0.7)

# # Add a purple horizontal line at y = 0
# plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# # Add title, labels, and legend
# plt.title("Arm1: Euler Angles Adjusted by Mean SUC Angle")
# plt.xlabel("Trial ID")
# plt.ylabel("Adjusted Euler Angle")
# plt.legend()

# # Display the plot
# plt.show()


In [414]:
# # Plot 'Euler' values for 'hand_early' and 'hand_late', adjusted by the mean SUC angle for 'hand'
# plt.figure(figsize=(10, 6))

# # Scatter plot for hand_early
# plt.scatter(arm2_early['Trial ID'], arm2_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][4], label='Arm2 Early', color='blue', alpha=0.7)

# # # Scatter plot for hand_late
# # plt.scatter(hand_late['Trial ID'], hand_late['Euler'] - mean_suc_df['Mean_SUC_Angle'][2], label='Hand Late', color='green', alpha=0.7)

# # Add a purple horizontal line at y = 0
# plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# # Add title, labels, and legend
# plt.title("Arm2: Euler Angles Adjusted by Mean SUC Angle")
# plt.xlabel("Trial ID")
# plt.ylabel("Adjusted Euler Angle")
# plt.legend()

# # Display the plot
# plt.show()


In [415]:
# # Plot 'Euler' values for 'foearm_early' and 'foearm_late', adjusted by the mean SUC angle for 'foearm'
# plt.figure(figsize=(10, 6))

# # Scatter plot for foearm_early
# plt.scatter(arm1_early['Trial ID'], arm1_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][3], label='arm1 Early', color='blue', alpha=0.7)

# # # Scatter plot for foearm_late
# # plt.scatter(foearm_late['Trial ID'], foearm_late['Euler'] - mean_suc_df['Mean_SUC_Angle'][0], label='Forearm Late', color='green', alpha=0.7)

# # Add a purple horizontal line at y = 0
# plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# # Add title, labels, and legend
# plt.title("Arm1: Euler Angles Adjusted by Mean SUC Angle")
# plt.xlabel("Trial ID")
# plt.ylabel("Adjusted Euler Angle")
# plt.legend()

# # Display the plot
# plt.show()

In [416]:
# # Plot 'Euler' values for 'foearm_early' and 'foearm_late', adjusted by the mean SUC angle for 'foearm'
# plt.figure(figsize=(10, 6))

# # Scatter plot for foearm_early
# plt.scatter(foearm_early['Trial ID'], foearm_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][0], label='Forearm Early', color='blue', alpha=0.7)

# # # Scatter plot for foearm_late
# # plt.scatter(foearm_late['Trial ID'], foearm_late['Euler'] - mean_suc_df['Mean_SUC_Angle'][0], label='Forearm Late', color='green', alpha=0.7)

# # Add a purple horizontal line at y = 0
# plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# # Add title, labels, and legend
# plt.title("Forearm: Euler Angles Adjusted by Mean SUC Angle")
# plt.xlabel("Trial ID")
# plt.ylabel("Adjusted Euler Angle")
# plt.legend()

# # Display the plot
# plt.show()

In [417]:
# combined_joints_early = hand_early['Euler']+elbow_early['Euler']+foearm_early['Euler'] 
# # combined_joints_late = hand_late['Euler']+elbow_late['Euler']+foearm_late['Euler'] 
# combined_mean_suc = mean_suc_df['Mean_SUC_Angle'][3]+mean_suc_df['Mean_SUC_Angle'][4]+mean_suc_df['Mean_SUC_Angle'][5]

# # Plot 'Euler' values for 'hand_early' and 'hand_late', adjusted by the mean SUC angle for 'hand'
# plt.figure(figsize=(10, 6))

# # Scatter plot for hand_early
# plt.scatter(hand_early['Trial ID'], combined_joints_early- combined_mean_suc, label='Hand Early', color='blue', alpha=0.7)

# # # Scatter plot for hand_late
# # plt.scatter(hand_late['Trial ID'],  combined_joints_late  - combined_mean_suc, label='Hand Late', color='green', alpha=0.7)

# # Add a purple horizontal line at y = 0
# plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# # Add title, labels, and legend
# plt.title("Combined Euler Angles Adjusted by Mean SUC Angle")
# plt.xlabel("Trial ID")
# plt.ylabel("Adjusted Euler Angle")
# plt.legend()

# # Display the plot
# plt.show()

In [423]:
# Plot 'Euler' values for 'foearm_early' and 'foearm_late', adjusted by the mean SUC angle for 'foearm'
plt.figure(figsize=(10, 6))

# Scatter plot for foearm_early
plt.scatter(hand_early['Trial ID'], abs(hand_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][2]), label='hand', color='blue', alpha=0.7)
# plt.scatter(elbow_early['Trial ID'], abs(elbow_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][1]), label='elbow', color='purple', alpha=0.7)
plt.scatter(arm1_early['Trial ID'], abs(arm1_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][3]), label='arm1', color='green', alpha=0.7)

# Add a purple horizontal line at y = 0
plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# Add title, labels, and legend
plt.title("Euler Angles Adjusted by Mean SUC Angle")
plt.xlabel("Trial ID in Rotation")
plt.ylabel("Adjusted Euler Angle (degree)")
plt.legend()

# Display the plot
plt.show()

In [422]:
# Plot 'Euler' values for 'foearm_early' and 'foearm_late', adjusted by the mean SUC angle for 'foearm'
plt.figure(figsize=(10, 6))

# Scatter plot for foearm_early
plt.scatter(foearm_early['Trial ID'], hand_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][2], label='hand', color='blue', alpha=0.7)
# plt.scatter(elbow_early['Trial ID'], elbow_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][1], label='elbow', color='purple', alpha=0.7)
plt.scatter(arm1_early['Trial ID'], arm1_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][3], label='arm1', color='green', alpha=0.7)

# Add a purple horizontal line at y = 0
plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# Add title, labels, and legend
plt.title("Euler Angles Adjusted by Mean SUC Angle")
plt.xlabel("Trial ID in Rotation")
plt.ylabel("Adjusted Euler Angle (degree)")
plt.legend()

# Display the plot
plt.show()

In [440]:
# Plot 'Euler' values for 'foearm_early' and 'foearm_late', adjusted by the mean SUC angle for 'foearm'
plt.figure(figsize=(10, 6))

# Smooth the SEM using a rolling window average
window_size = 10  # Adjust the window size as needed

hand_error = abs(hand_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][2])
shoulder_error = abs(arm1_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][3])
smoothed_hand_error = hand_error.rolling(window=window_size, min_periods=1, center=True).mean()
smoothed_shoulder_error= shoulder_error.rolling(window=window_size, min_periods=1, center=True).mean()


# Scatter plot for foearm_early
plt.plot(hand_early['Trial ID'], smoothed_hand_error, label='hand mean', color='blue', alpha=0.7, linewidth=2)
plt.scatter(hand_early['Trial ID'], hand_error, label='hand', color='blue', alpha=0.5, linewidth=0.3)

# plt.scatter(elbow_early['Trial ID'], abs(elbow_early['Euler'] - mean_suc_df['Mean_SUC_Angle'][1]), label='elbow', color='purple', alpha=0.7)
plt.fill_between(hand_early['Trial ID'], 0.5*smoothed_hand_error, 1.5*smoothed_hand_error, color='blue',alpha=0.3)


plt.plot(arm1_early['Trial ID'], smoothed_shoulder_error, label='shoulder mean', color='green', alpha=0.7, linewidth=2)
plt.scatter(arm1_early['Trial ID'], shoulder_error, label='shoulder', color='green', alpha=0.5, linewidth=0.3)

plt.fill_between(hand_early['Trial ID'], 0.5*smoothed_shoulder_error, 1.5*smoothed_shoulder_error, color='green',alpha=0.3)
# # Add a purple horizontal line at y = 0
# plt.axhline(y=0, color='purple', linestyle='--', linewidth=1.5)

# Add title, labels, and legend
plt.title("Euler Angles Adjusted by Mean SUC Angle")
plt.xlabel("Trial ID in Rotation")
plt.ylabel("Error (degree)")
plt.legend()

# Display the plot
plt.savefig('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/plot/subject3.png', bbox_inches='tight', dpi=600)